In [7]:
import pandas as pd
from statsmodels.tsa.filters.hp_filter import hpfilter

# 1) 동행종합지수(2020=100) 읽기
df = (pd.read_csv("..\data\경기국면\경기종합지수_동행종합지수.csv")
        .rename(columns=str.strip)
        .assign(날짜=lambda d: pd.to_datetime(d['날짜']))
        .set_index('날짜'))

# 2) HP 필터(λ=129,600) → 순환변동치
cycle, _ = hpfilter(df['동행종합지수(2020=100)'], lamb=129_600)
df['cycle'] = cycle

# 3) 3개월 이동평균으로 소음 제거
df['cycle_ma3'] = df['cycle'].rolling(3, center=False).mean()

# 4) 월별 모멘텀(전월 대비 증감) 계산
df['Δ'] = df['cycle_ma3'].diff()

# 5) 규칙 적용
cond_expand = (
    (df['cycle_ma3'] > 0) &                # 0선 위
    (df['Δ'].rolling(2).apply(lambda x: (x > 0).all(), raw=True))  # 2개월 연속 상승
)
cond_contract = (
    (df['cycle_ma3'] < 0) &
    (df['Δ'].rolling(2).apply(lambda x: (x < 0).all(), raw=True))  # 2개월 연속 하락
)

df['regime'] = pd.NA
df.loc[cond_expand,   'regime'] = 1   # 확장
df.loc[cond_contract, 'regime'] = 0   # 수축

# 6) 앞쪽·중간 공백을 모두 채운 뒤 int로 변환
df['regime'] = (
    df['regime']
      .ffill()     # 이전 값으로 채움
      .bfill()     # 그래도 NA면 뒤의 첫 값으로 채움
      .astype(int) # 이제 안전하게 0/1 정수형
)

# 결과 확인
print(df[['cycle_ma3', 'regime']].tail(12))

            cycle_ma3  regime
날짜                           
2024-01-01   0.494216       1
2024-02-01   0.444447       1
2024-03-01   0.361789       1
2024-04-01   0.246193       1
2024-05-01  -0.002392       0
2024-06-01  -0.184020       0
2024-07-01  -0.532079       0
2024-08-01  -0.779953       0
2024-09-01  -0.994357       0
2024-10-01  -0.942000       0
2024-11-01  -1.022917       0
2024-12-01  -1.137133       0


<>:5: SyntaxWarning: invalid escape sequence '\d'
<>:5: SyntaxWarning: invalid escape sequence '\d'
C:\Users\user\AppData\Local\Temp\ipykernel_5836\256907310.py:5: SyntaxWarning: invalid escape sequence '\d'
  df = (pd.read_csv("..\data\경기국면\경기종합지수_동행종합지수.csv")
C:\Users\user\AppData\Local\Temp\ipykernel_5836\256907310.py:38: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  .bfill()     # 그래도 NA면 뒤의 첫 값으로 채움


In [8]:
df[['cycle_ma3', 'regime']]

,cycle_ma3,regime
날짜,,
2015-01-01,NaN,0
2015-02-01,NaN,0
2015-03-01,-0.662371,0
2015-04-01,-0.628779,0
2015-05-01,-0.861836,0
...,...,...
2024-08-01,-0.779953,0
2024-09-01,-0.994357,0
2024-10-01,-0.942000,0


In [4]:
import pandas as pd
from statsmodels.tsa.filters.hp_filter import hpfilter

# 1) 원본 CSV 읽기 ─ 날짜 컬럼 파싱 & 인덱스화
PATH = "..\data\경기국면\경기종합지수_동행종합지수.csv"   # 필요 시 절대경로로 수정
df = (pd.read_csv(PATH)
        .rename(columns=str.strip)         # 혹시 모를 공백 제거
        .assign(날짜=lambda d: pd.to_datetime(d['날짜']))
        .set_index('날짜'))

# 2) HP-filter(λ = 129 600) → 순환변동치 계산
cycle, _ = hpfilter(df['동행종합지수(2020=100)'], lamb=129_600)
df['동행지수_순환변동치'] = cycle       # 새로운 컬럼 추가

# 3) 월(月) 단위 결과 저장
df[['동행지수_순환변동치']].to_csv("CCI_cycle_monthly.csv", encoding="utf-8-sig")

# --- (선택) 일(日) 단위로 펼치고 싶을 때 --------------------------
daily_cycle = (df['동행지수_순환변동치']
                 .resample('D')   # 일간으로 리샘플링
                 .ffill())        # 각 달 값을 그 달 마지막 날까지 forward-fill
daily_cycle.to_csv("CCI_cycle_daily.csv", encoding="utf-8-sig")
# ---------------------------------------------------------------

print("✓ 변환 완료: CCI_cycle_monthly.csv / CCI_cycle_daily.csv 저장")

✓ 변환 완료: CCI_cycle_monthly.csv / CCI_cycle_daily.csv 저장


<>:5: SyntaxWarning: invalid escape sequence '\d'
<>:5: SyntaxWarning: invalid escape sequence '\d'
C:\Users\user\AppData\Local\Temp\ipykernel_5836\3596884972.py:5: SyntaxWarning: invalid escape sequence '\d'
  PATH = "..\data\경기국면\경기종합지수_동행종합지수.csv"   # 필요 시 절대경로로 수정


In [9]:
data = pd.read_csv(r'..\data\경기국면\CCI_cycle_regimes_full.csv')
data

,날짜,cycle_raw,regime_raw,cycle_ma3,regime_smooth
0,2015-01-01,-0.829289,0,-0.662371,0
1,2015-02-01,-0.462373,0,-0.662371,0
2,2015-03-01,-0.695451,0,-0.662371,0
3,2015-04-01,-0.728513,0,-0.628779,0
4,2015-05-01,-1.161543,0,-0.861836,0
...,...,...,...,...,...
115,2024-08-01,-1.027721,0,-0.779953,0
116,2024-09-01,-1.042020,0,-0.994357,0
117,2024-10-01,-0.756260,0,-0.942000,0
118,2024-11-01,-1.270470,0,-1.022917,0


In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120 entries, 0 to 119
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   날짜             120 non-null    object 
 1   cycle_raw      120 non-null    float64
 2   regime_raw     120 non-null    int64  
 3   cycle_ma3      120 non-null    float64
 4   regime_smooth  120 non-null    int64  
dtypes: float64(2), int64(2), object(1)
memory usage: 4.8+ KB


In [22]:
df1 = data[['날짜', 'cycle_raw', 'regime_raw']].copy()
df1['날짜'] = pd.to_datetime(df1['날짜'])
# '날짜' 컬럼을 년-월 형식으로 변환
df1['날짜'] = df1['날짜'].dt.strftime('%Y-%m')
df1.rename(columns={'날짜': '상장일'}, inplace=True)

df2 = data[['날짜', 'cycle_ma3', 'regime_smooth']].copy()
df2['날짜'] = pd.to_datetime(df2['날짜'])
df2['날짜'] = df2['날짜'].dt.strftime('%Y-%m')
df2.rename(columns={'날짜': '상장일'}, inplace=True)

In [23]:
giup = pd.read_csv(r'..\data\giup_data.csv')
# 상장일 컬럼 datetime 형식으로 변환
giup['상장일'] = pd.to_datetime(giup['상장일'])
# '상장일' 컬럼을 년-월 형식으로 변환
giup['상장일'] = giup['상장일'].dt.strftime('%Y-%m')

In [24]:
giup = giup[['회사명','상장일']]

In [26]:
pd.merge(giup, df1, on='상장일', how='left').to_csv(r'..\data\경기국면\동행순환변동치_raw.csv', index=False, encoding='utf-8-sig')
pd.merge(giup, df2, on='상장일', how='left').to_csv(r'..\data\경기국면\동행순환변동치_smooth.csv', index=False, encoding='utf-8-sig')